In [1]:
%matplotlib widget
%config InlineBackend.figure_format = "svg"

import addict
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm
from typing import List, Dict, Tuple

import celeri
celeri = reload(celeri)

plt.rcParams["text.usetex"] = False # Plotting the global model is much much faster with tex fonts turned off

# Read in data files and do basic processing

In [2]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)

In [ ]:
# Finding the edges of meshes
for i in range(len(meshes)):
        # Make side arrays containing vertex indices of sides
        vertices = meshes[i].verts
        side_1 = np.sort(np.vstack((vertices[:, 0], vertices[:, 1])).T, 1)
        side_2 = np.sort(np.vstack((vertices[:, 1], vertices[:, 2])).T, 1)
        side_3 = np.sort(np.vstack((vertices[:, 2], vertices[:, 0])).T, 1)
        all_sides = np.vstack((side_1, side_2, side_3))
        unique_sides, sides_count = np.unique(all_sides, return_counts=True, axis=0)
        edge_nodes = unique_sides[np.where(sides_count == 1)]

        meshes[i].ordered_edge_nodes = np.zeros_like(edge_nodes)
        meshes[i].ordered_edge_nodes[0, :] = edge_nodes[0, :]
        last_row = 0
        for j in range(1, len(edge_nodes)):
            idx = np.where((edge_nodes==meshes[i].ordered_edge_nodes[j-1, 1])) # Edge node indices the same as previous row, second column
            next_idx = np.where(idx[0][:] != last_row) # One of those indices is the last row itself. Find the other row index
            next_row = idx[0][next_idx] # Index of the next ordered row
            next_col = idx[1][next_idx] # Index of the next ordered column (1 or 2)
            if next_col == 1:
                next_col_ord = [1, 0] # Flip edge ordering
            else:
                next_col_ord = [0, 1]
            meshes[i].ordered_edge_nodes[j, :] = edge_nodes[next_row, next_col_ord] 
            last_row = next_row # Update last_row so that it's excluded in the next iteration
plt.figure(figsize=(12,7))
plt.plot(meshes[i].meshio_object.points[meshes[i].ordered_edge_nodes[:, 0], 0], meshes[i].meshio_object.points[meshes[i].ordered_edge_nodes[:, 0], 1])
print(meshes[i].ordered_edge_nodes)

# Create storage dictionaries and calculate elastic operators

In [3]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this work correctly

# Elastic operators and TDE smoothing
# TODO: Add n_tde_dims to get_all_mesh_smoothing_matrices
operators.slip_rate_to_okada_to_velocities = celeri.get_segment_station_operator_okada(segment, station, command)

# TODO: Rename to tde_to_velocities
# TODO: There should be one tde_to_velocities per mesh
# so we should have something like operators.meshes[0].tde_to_velocities
# TODO: Move smoothing matrices to operators.meshes[0].smoothing_matrix_name
operators.tri_station = celeri.get_tri_station_operator_okada(meshes, station, command)
celeri.get_all_mesh_smoothing_matrices(meshes)
celeri.get_all_mesh_smoothing_matrices_simple(meshes, 2)

Calculating Okada partials for segments:   0%|          | 0/837 [00:00<?, ?it/s]

Calculating cutde partials for triangles:   0%|          | 0/1841 [00:00<?, ?it/s]

# Calculate additional non-elastic operators

In [4]:
# Build all linear kinematic operators
operators.rotation_to_velocities = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)

# TODO: Update celeri.block_constraints to properly use block.
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.rotation_to_slip_rate = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)  # TODO: Rename block_strain_rate_to_velocities
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)  # TODO: Rename mogi_to_velocities

In [5]:
# TODO: Move all of this down to the cell with matrix assembly
# TODO: Eliminate creation of tde_matrix
# TODO: Loop over each mesh
# Shrink operators.tri_station so that there are no vertical displacments and no tensile slip
tde_matrix = copy.deepcopy(operators.tri_station)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[0], 3), axis=0)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[1], 3), axis=1)

# Slip estimation with direct inverse and smoothing matrix
# TODO: Each mesh should have a smoothing weight in .json file
# TODO: Loop over each mesh
# TODO: Eliminate copy to smoothing matrix
smoothing_matrix = 1e2 * meshes[0].smoothing_matrix.toarray()  # TODO: Do we need to keep this coefficient since we use meshes[0].smoothing_weight below?
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[0], 3), axis=0)
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[1], 3), axis=1)
smoothing_matrix = meshes[0].smoothing_weight * smoothing_matrix # Weight smoothing matrix
tde_and_smoothing_matrix = np.vstack((tde_matrix, smoothing_matrix))

# Sketching out the assembly of the block model system
The list here is in as of right now (09/19/21)...a work in progress

## The observation vector:    
| What | Symbolically | Variable name |
| - | - | - |
| GPS velocities | $\mathbf{v}$ | ```assembly.data.east_vel, assembly.data.north_vel``` |
| Slip rate constraints | $\dot{\mathbf{s}}$ | ```assembly.slip_rate_constraints``` |
| Rotation vector constraints | $\boldsymbol{\omega}$ | ```assembly.block_constraints``` |

## The state vector:    
| What | Symbolically | Variable name |
| - | - | - |
| Block rotation rates | $\boldsymbol{\Omega}$ | ```estimation.block_rotation_vectors``` |
| TDE slip | $\mathbf{t}$ | ```estimation.tri_slip_rates``` |
  
## Matrix operators:    
| What | Symbolically | Variable name |
| - | - | - |
| Rotation vector to GPS | $\boldsymbol{\Omega} \rightarrow \mathbf{v}$ | ```operators.block_rotation``` | 
| Rotation vector to GPS | $\boldsymbol{\Omega} \rightarrow \mathbf{v}$ | ```operators.global_float_block_rotation``` | 
| Rotation vector to segment slip rates| $\boldsymbol{\Omega} \rightarrow \dot{\mathbf{s}}$ | ```operators.slip_rate_segment_block``` | 
| Segment slip rates to elastic deformation | $\dot{\mathbf{s}} \rightarrow \mathbf{v}$ | ```operators.okada_segment_station``` | 
| TDE slip to GPS | $\mathbf{t} \rightarrow \mathbf{v}$ | ```operators.tri_station``` |
| Rotation vector to slip rate constraints| $\boldsymbol{\Omega} \rightarrow \dot{\mathbf{s}}$ | ```operators.slip_rate_constraints``` |
| Rotation vector to Euler pole contraints | $\mathbf{I}$ | ```operators.block_motion_contraints``` |



# Block model with block rotations, fully locked segments and partially locked subduction zone using the full tde_matrix and smoothing matrix

In [6]:
# TODO: What should this converstion be?
DEG_PER_MYR_TO_RAD_PER_YR = np.deg2rad(1) / 1e6

n_tde = int(tde_matrix.shape[1] / 2)
n_stations = assembly.data.n_stations
n_blocks = len(block)
n_block_constraints = assembly.data.n_block_constraints

idx = addict.Dict()
idx.start_station_row = 0
idx.end_station_row = 2 * len(station)
idx.start_block_col = 0
idx.end_block_col = len(block)
idx.start_tde_col = idx.end_block_col
idx.end_tde_col = idx.start_tde_col + n_tde
idx.meshes[0].start_tde_col = 0
idx.meshes[0].end_tde_col = 0
idx.meshes[0].start_tde_eigs_col = 0
idx.meshes[0].end_tde_eig_col = 0

# Build data vector
data_vector = np.zeros(2 * n_stations + 2 * n_tde + 3 * n_block_constraints)
data_vector[0:2 * n_stations] = celeri.interleave2(assembly.data.east_vel, assembly.data.north_vel) # add GPS data
data_vector[2 * n_stations + 2 * n_tde:2 * n_stations + 2 * n_tde + 3 * n_block_constraints] = DEG_PER_MYR_TO_RAD_PER_YR * assembly.data.block_constraints # add plate motion constraints Not sure if units are correct

# Delete rows associated with vertical velocities
operators.slip_rate_to_okada_to_velocities = np.delete(operators.slip_rate_to_okada_to_velocities, np.arange(2, operators.slip_rate_to_okada_to_velocities.shape[0], 3), axis=0)
operators.rotation_to_slip_rate_to_okada_to_velocities = operators.slip_rate_to_okada_to_velocities @ operators.rotation_to_slip_rate
operators.rotation_to_velocities = np.delete(operators.rotation_to_velocities, np.arange(2, operators.rotation_to_velocities.shape[0], 3), axis=0)


# Print sizes of matrices that we'll use to build operator
print("data_vector: " + str(data_vector.shape))
print("operators.block_motion_constraints.shape: " + str(operators.block_motion_constraints.shape))
print("operators.rotation_to_velocities.shape: " + str(operators.rotation_to_velocities.shape))
print("rotation_to_slip_rate_to_okada_to_velocities.shape: " + str(operators.rotation_to_slip_rate_to_okada_to_velocities.shape))
print("tde_matrix.shape: " + str(tde_matrix.shape))
print("smoothing_matrix.shape: " + str(smoothing_matrix.shape))
print("tde_and_smoothing_matrix.shape: " + str(tde_and_smoothing_matrix.shape))
print("n_tde: " + str(n_tde))

# Initialize linear operator
operator = np.zeros((2 * assembly.data.n_stations # start rows
                     + 2 * n_tde
                     + 3 * assembly.data.n_block_constraints,
                       3 * len(block) # start columns
                     + 2 * n_tde))
print("operator.shape: " + str(operator.shape))
print("Finished printing")

# Insert block rotation and fully coupled segments
operator[0:2 * n_stations, 0:3 * n_blocks] = operators.rotation_to_velocities - operators.rotation_to_slip_rate_to_okada_to_velocities

# Insert block motion constraints
operator[2 * n_stations + 2 * n_tde:2 * n_stations + 2 * n_tde + 3 * n_block_constraints, 0:3 * n_blocks] = operators.block_motion_constraints

# Insert tdes and smoothing matrix
print(operator.shape)
print(2 * n_stations + 2 * n_tde)
print(3 * n_blocks + 2 * n_tde)
print(operator[0:2 * n_stations + 2 * n_tde, 3 * n_blocks : 3 * n_blocks + 2 * n_tde].shape)
print(tde_and_smoothing_matrix.shape)
operator[0:2 * n_stations + 2 * n_tde, 3 * n_blocks : 3 * n_blocks + 2 * n_tde] = tde_and_smoothing_matrix

# Solve the overdetermined linear system
# Overdetermined state estimation
state_covariance_matrix = np.linalg.inv(operator.T @ operator)
state_vector = state_covariance_matrix @ operator.T @ data_vector 

# TODO: Move all of the estimation postprocessing to a function process_estimation()?
estimation = addict.Dict()
estimation.predictions = operator @ state_vector
estimation.vel = estimation.predictions[0:2 * n_stations]
estimation.east_vel = estimation.vel[0::2]
estimation.north_vel = estimation.vel[1::2]

# Estimate slip rate uncertainties
estimation.slip_rate_sigma = np.sqrt(np.diag(operators.rotation_to_slip_rate @ state_covariance_matrix[0:3 * n_blocks, 0:3 * n_blocks] @ operators.rotation_to_slip_rate.T)) # I don't think this is correct because for the case when there is a rotation vector a priori
estimation.strike_slip_rate_sigma = estimation.slip_rate_sigma[0::3]
estimation.dip_slip_rate_sigma = estimation.slip_rate_sigma[1::3]
estimation.tensile_slip_rate_sigma = estimation.slip_rate_sigma[2::3]

# Calculate mean squared residual velocity
estimation.east_vel_residual = estimation.east_vel - station.east_vel
estimation.north_vel_residual = estimation.north_vel - station.north_vel

# Extract TDE slip rates from state vector
estimation.tde_rates = state_vector[3 * n_blocks : 3 * n_blocks + 2 * n_tde]
estimation.tde_strike_slip_rates = estimation.tde_rates[0::2]
estimation.tde_dip_slip_rates = estimation.tde_rates[1::2]

# Extract segment slip rates from state vector
estimation.slip_rates = operators.rotation_to_slip_rate @ state_vector[0:3 * n_blocks]
estimation.strike_slip_rates = estimation.slip_rates[0::3]
estimation.dip_slip_rates = estimation.slip_rates[1::3]
estimation.tensile_slip_rates = estimation.slip_rates[2::3]

# Calculate rotation only velocities
estimation.vel_rotation = operators.rotation_to_velocities @ state_vector[0:3 * n_blocks]
estimation.east_vel_rotation = estimation.vel_rotation[0::2]
estimation.north_vel_rotation = estimation.vel_rotation[1::2]

# Calculate fully locked segment velocities
estimation.vel_elastic_segment = operators.rotation_to_slip_rate_to_okada_to_velocities @ state_vector[0:3 * n_blocks]
estimation.east_vel_elastic_segment = estimation.vel_elastic_segment[0::2]
estimation.north_vel_elastic_segment = estimation.vel_elastic_segment[1::2]

# Calculate TDE velocities
estimation.vel_tde = tde_matrix @ state_vector[3 * n_blocks:]
estimation.east_vel_tde = estimation.vel_tde[0::2]
estimation.north_vel_tde = estimation.vel_tde[1::2]

data_vector: (7057,)
operators.block_motion_constraints.shape: (3, 93)
operators.rotation_to_velocities.shape: (3372, 93)
rotation_to_slip_rate_to_okada_to_velocities.shape: (3372, 93)
tde_matrix.shape: (3372, 3682)
smoothing_matrix.shape: (3682, 3682)
tde_and_smoothing_matrix.shape: (7054, 3682)
n_tde: 1841
operator.shape: (7057, 3775)
Finished printing
(7057, 3775)
7054
3775
(7054, 3682)
(7054, 3682)


# Plot model summary 

In [7]:
celeri.plot_estimation_summary(segment, station, estimation, lon_range=(225, 250), lat_range=(30, 52), quiver_scale=1e2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.figure()
plt.plot(np.log10(np.abs(state_vector[0:3 * n_blocks])), "r+")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …